In [1]:
import glob
import os
import numpy as np
import pandas as pd
import sys

from collections import OrderedDict

from matplotlib.patches import Rectangle
from matplotlib import pyplot as plt
# %matplotlib qt5

from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
from scipy.stats import f_oneway
from scipy.stats import normaltest



import config 
sys.path.append('./models/v1model/')
from plotting import _get_barplot
from utils import turn_tex
turn_tex('on')

path = '/run/media/loaloa/lbb_ssd/primitives/processed2/results/'
%matplotlib qt5

In [2]:

data = pd.read_csv(f'{path}/all_primitives_results.csv', index_col=[0,1,2])


In [17]:
def compute_singles(group_values, group_levels):
    mwu_pvalues = []
    for x, x_name in zip(group_values, group_levels):
        mwu_pvals = []
        for y, y_name in zip(group_values, group_levels):
            mwu_pvals.append(mannwhitneyu(x, y).pvalue)
            # print(f'{x_name}, {y_name}: p: {p:.4f}')
        mwu_pvalues.append(mwu_pvals)
    mwu_pvalues = np.array(mwu_pvalues)

    fig, ax = plt.subplots()
    fig.subplots_adjust(bottom=.2)
    im = ax.imshow(mwu_pvalues, vmin=0, vmax=.3)
    plt.scatter(*np.where(mwu_pvalues<0.05), marker='*', color='w', s=100)
    plt.colorbar(im, orientation='vertical', label='p-value MWU test')
    plt.title(exp_name)

    lbls = [l[0] for l in group_levels]
    ax.set_yticklabels((lbls))
    ax.set_yticks(np.arange(len(lbls)))
    ax.set_xticklabels((lbls), rotation=45, ha='right')
    ax.set_xticks(np.arange(len(lbls)))



def test_kruskal_wallis(exp_data, factor, parametric=False, passed_designs=None):
    
    designs = [[d] for d in exp_data.index.unique('design')]
    if passed_designs is not None:
        designs = passed_designs
    
    groups = [exp_data.loc[d].stack().dropna() for d in designs if len(exp_data.loc[d].stack().dropna()) >2]
    group_levels = [g.name for g in groups]
    group_values = [g.values for g in groups]

    if not parametric:
        t_stat, p = kruskal(*group_values)
        which_test = 'Kruskal Wallis'
    else:
        t_stat, p = f_oneway(*group_values)
        which_test = 'ANOVA'
        var_homog = sorted([np.var(gv) for gv in group_values])
        var_homog = max(var_homog)/min(var_homog)
        normal = [normaltest(gv).pvalue if len(gv) > 7 else 0 for gv in group_values]
        normal = [p>.05 if p != 0 else 'too few samples' for p in normal]
    
    sig = '***' if p <.05 else 'N.S.'
    lbl = f'{config.SPACER}\n{factor}: n levels:{len(designs)}:\n {which_test} t={t_stat:.3f}, p={p:.4f}\n --> {sig}\n\n'
    if parametric:
        print(f'Within group normal distr: {normal}')
        print(f'Variance homogeneity (max(group_var)/min(group_var)): {var_homog}')


    print(lbl)
    
    # if passed_designs is None:
    compute_singles(group_values, designs)
    
            
exp_names = data.index.unique('exp_name')
div = 7
for exp_name in exp_names:
    if div in (7,14):
        exp_dat = data.loc[(exp_name, div),:]
    elif div in ('split', 'merge'):
        exp_dat = data.loc[exp_name,:]
        if div == 'merge':
            exp_dat = exp_dat.unstack('DIV').T.reset_index(drop=True).T
    exp_name = f'{exp_name} DIV:{div}'


    designs = None
    if exp_name.startswith('Edge transition'):
        exp_name += '_radius'
        designs = (
        ['2um, 15um','3.5um, 15um','5.5um, 15um',],
        ['2um, 25um','3.5um, 25um','5.5um, 25um',],
        ['2um, inf','3.5um, inf','5.5um, inf',], 
        ['neg. control'],)
        p = test_kruskal_wallis(exp_dat, exp_name, passed_designs=designs)
        
        exp_name += '_opening'
        designs = (
        ['5.5um, 15um','5.5um, 25um','5.5um, inf',], 
        ['3.5um, 15um','3.5um, 25um','3.5um, inf',],
        ['2um, 15um','2um, 25um','2um, inf',],
        ['neg. control'],)
        p = test_kruskal_wallis(exp_dat, exp_name, passed_designs=designs)
        
    elif exp_name.startswith('Growth speed'):
        exp_name += '_chnl_height_comparison'

        designs = [['W:0.5um - H:6um', 'W:0.75um - H:6um', 'W:1um - H:6um', 'W:1.25um - H:6um', 'W:1.5um - H:6um', 'W:1.75um - H:6um', 'W:2um - H:6um', 'W:3um - H:6um', 'W:4um - H:6um', 'W:6um - H:6um', 'W:8um - H:6um', 'W:10um - H:6um', 'W:14um - H:6um', 'W:18um - H:6um', 'W:22um - H:6um', 'W:26um - H:6um',],
    ['W:0.5um - H:24um', 'W:0.75um - H:24um', 'W:1um - H:24um', 'W:1.25um - H:24um', 'W:1.5um - H:24um', 'W:1.75um - H:24um', 'W:2um - H:24um', 'W:3um - H:24um', 'W:4um - H:24um', 'W:6um - H:24um', 'W:8um - H:24um', 'W:10um - H:24um', 'W:14um - H:24um', 'W:18um - H:24um', 'W:22um - H:24um', 'W:26um - H:24um',]]
    elif exp_name.startswith('Directionality constrain'):
        backw_designs = ['backward rescue loops',
                    'backward rescue loops angled',
                    'backward stomache',
                    'backward negative control']
        p = test_kruskal_wallis(exp_dat.loc[backw_designs], 'Backw.' + exp_name, passed_designs=None)
        forw_designs = ['forward rescue loops',
                    'forward rescue loops angled',
                    'forward stomache',
                    'forward negative control']
        p = test_kruskal_wallis(exp_dat.loc[forw_designs], 'Forw.' + exp_name, passed_designs=None)

    else:
        p = test_kruskal_wallis(exp_dat, exp_name, passed_designs=designs)
    
    


Cue gradient DIV:7: n levels:11:
 Kruskal Wallis t=10.649, p=0.3855
 --> N.S.


Backw.Directionality constrain DIV:7: n levels:4:
 Kruskal Wallis t=1.776, p=0.6202
 --> N.S.


Forw.Directionality constrain DIV:7: n levels:4:
 Kruskal Wallis t=0.869, p=0.8329
 --> N.S.


Edge transition DIV:7_radius: n levels:4:
 Kruskal Wallis t=5.887, p=0.1172
 --> N.S.


Edge transition DIV:7_radius_opening: n levels:4:
 Kruskal Wallis t=7.609, p=0.0548
 --> N.S.


Radial detachment DIV:7: n levels:5:
 Kruskal Wallis t=7.073, p=0.1321
 --> N.S.


Edge attachment DIV:7: n levels:8:
 Kruskal Wallis t=9.814, p=0.1994
 --> N.S.


Joining channels DIV:7: n levels:8:
 Kruskal Wallis t=7.458, p=0.2806
 --> N.S.


